# S&P 500 Analysis

## Import the Packages, Functions and Data

In [ ]:
import pandas as pd
from EDA_functions import EDA, normality_check,stationarity_check,decomposition_plot,daily_returns
from Baseline_functions import capital_calculation, calculate_macd_signals, profit_trades, loss_trades
from DQN_functions import create_states, ReplayMemory, DQNAgent,train_agent,evaluate_agent, ConvDQN,plot_training,create_action_episode_df
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_pickle('data/SP500.pkl')

## EDA

### Visual

In [ ]:
EDA(df)

In [ ]:
df.describe()

### Without Differencing

In [ ]:
decomposition_plot(df,0)

In [ ]:
stationarity_check(df,0)

In [ ]:
normality_check(df,0)

### After 1st Difference

In [ ]:
decomposition_plot(df,1)

In [ ]:
stationarity_check(df,1)

In [ ]:
normality_check(df,1)

### Outlier Check

Since price gradually increases, it does not make sense to use traditional outlier checking methods. Therefore we calculate the daily change and then the outliers in those

In [ ]:
daily_returns(df)

## Outlier Removal 


In [ ]:
df['daily_return'] = df['Close'].pct_change()
df['daily_return'] = df['daily_return'].fillna(0)
#mean plus 3sigma value for absolute daily returns
mean = df['daily_return'].mean()
std = df['daily_return'].std()
upper_limit = mean + 2*std
lower_limit = mean - 2*std
print(f'Mean: {mean}, Std: {std}, Upper Limit: {upper_limit}, Lower Limit: {lower_limit}')
#number of rows that are outside the 3 sigma range
print(f'Number of rows outside 3 sigma range: {len(df[(df["daily_return"]>upper_limit) | (df["daily_return"]<lower_limit)])}')
print(f'Percentage of rows outside 3 sigma range: {len(df[(df["daily_return"]>upper_limit) | (df["daily_return"]<lower_limit)])/len(df)*100}% ')

#if daily return is outside 3 sigma range, replace all the values with previous day's values
for i in range(1,len(df)):
    if df['daily_return'][i]>upper_limit or df['daily_return'][i]<lower_limit:
        df['Close'][i] = df['Close'][i-1]
        df['Open'][i] = df['Open'][i-1]
        df['High'][i] = df['High'][i-1]
        df['Low'][i] = df['Low'][i-1]
        df['Volume'][i] = df['Volume'][i-1]
        df['daily_return'][i] = 'NaN'

## Baseline Models


In [ ]:
df_base = calculate_macd_signals(df)

In [ ]:
df_base = profit_trades(df_base)

In [ ]:
df_base = loss_trades(df_base)

## DQN

### Training

In [ ]:
df_base = df[['Open','High','Low','Close','Volume']]

#first 80% of the data is train
df_train = df_base.iloc[:int(len(df_base)*0.8)]
#last 20% of the data is test
df_test = df_base.iloc[int(len(df_base)*0.8):]

In [ ]:
#min-max scaling manually

df_train_scaled = (df_train - df_train.min())/(df_train.max()-df_train.min())
df_test_scaled = (df_test - df_train.min())/(df_train.max()-df_train.min())


In [ ]:
display(df_train.head())

display(df_test.head())

In [ ]:
print(df_train.shape, df_test.shape)
print(df_train.shape[0] + df_test.shape[0])
print(df.shape[0])

In [ ]:
window_size = 9
states = create_states(df_train_scaled, window_size)
test_states = create_states(df_test_scaled, window_size)
input_dim_conv = states.shape[2]
output_dim = 3
print("Shape of states:", states.shape)

In [ ]:
model = ConvDQN(input_dim_conv, output_dim, window_size)
memory = ReplayMemory(50000)
agent = DQNAgent(input_dim_conv, output_dim, window_size, lr=0.0001, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.999)

In [ ]:
states.shape[0]

In [ ]:
%%time
log_train = train_agent(agent, states, 350, batch_size=32)

In [ ]:
log_train.to_csv('SP500/log_train_CNN_adjusted_lr_hyp_tuned_350_ep_mu2sigma.csv', index=False)

In [ ]:
#save the model
torch.save(agent.model.state_dict(), 'SP500/dqn_model_log_train_CNN_adjusted_lr_hyp_tuned.pth')


### Evaluating and Testing

In [ ]:
log_train = pd.read_csv('SP500/log_train_CNN_adjusted learning rate.csv')

In [ ]:
plot_training(log_train)

In [ ]:
last_episode = log_train[log_train['Episode'] == log_train['Episode'].max()]

In [ ]:
last_episode.reset_index(drop=True, inplace=True)
last_episode.head()

In [ ]:
last_episode['Close'] = last_episode['Price'].to_list()

In [ ]:
last_episode['Capital_DQN'] = capital_calculation(last_episode,'Action')

In [ ]:
action_episode_df = create_action_episode_df(log_train)

In [ ]:
#get value counts of all columns in the DataFrame
action_episode_df.apply(pd.Series.value_counts).transpose()[['Buy']].plot()

In [ ]:
# Initialize the agent
test_state_size = test_states.shape[1] * test_states.shape[2]
action_size = 3
agent = DQNAgent(state_size, action_size, lr=0.00001, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.999)

In [ ]:
test_state_size

In [ ]:
model_path = 'SP500/dqn_modelSP500_CNN_500_wo_outliers_hyp_tuned.pth'
agent.model.load_state_dict(torch.load(model_path))

# Ensure the model is in evaluation mode
agent.model.eval()

In [ ]:
%%time
total_reward, actions = evaluate_agent(agent, test_states)
print(f"Total Reward on Test Data: {total_reward}")

### Hyperparameter Tuning

In [ ]:
# # Optimize hyperparameters with Optuna
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# print(f"Best hyperparameters: {study.best_params}")

##